In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import numpy as np
import pandas as pd
import re #Using regular expressions to search for special sequence of characters
import winsound
import time

# All arrays used in program 

In [ ]:
#These 2 arrays are used to store the webpages extracted through webscraping
IndustryLinks=np.empty(0)
CompaniesLinks=np.empty(0)

#This array stores the part of the name of industry used as keyword while parsing through html doc
IndustryNames=np.empty(0)

#These 2 arrays are used to store the names of industries and companies extracted
CompanyNames=np.empty(0)
AllIndustries=np.empty(0)

#Used to store the values of parameters that is extracted for each company

beta=np.empty(0)#contains beta values
pe=np.empty(0)#contains p/e values
div=np.empty(0)#contains dividend values
indpe=np.empty(0)#contains industry p/e values
pb=np.empty(0)#contains p/b values
divy=np.empty(0)#contains dividend yield values
deliv1d=np.empty(0)#contains 1-day deliverables
deliv3d=np.empty(0)#contains 3-day deliverables
deliv5d=np.empty(0)#contains 5-day deliverables
deliv8d=np.empty(0)#contains 8-day deliverables

df=pd.DataFrame(columns=['Industry','Company','Beta','P/E','Dividend(%)','Industry P/E','P/B','Dividend Yield(%)','Deliverables(1D)','Deliverables(3D)','Deliverables(5D)','Deliverables(8D)'])

# WebScraping

In [ ]:
def Beta(Clink):
    
    page=requests.get(Clink)       #Using companylink passed tocreate soup object
    soup=bs(page.content,'lxml')
    
    attr4={'class':'nsert','id':'div_nse_livebox_wrap'}           #Using attributes to locate nse section of 
    nseData=soup.find('div',attrs=attr4)                          #html doc
    beta=nseData.find_all('div',attrs={'class':'disin vt'})       #Finding the part of the code 
    
    for div in beta:
        temp=str([div.text])
        
    temp1=temp.split('\\n')
    #print(temp1[2])
    try:
        return float(temp1[2])
    except:
        return np.NaN

In [ ]:
def terms(Clink):
    try:
        temp=np.empty(0)
        temp1=np.empty(0)
        page=requests.get(Clink)
        soup=bs(page.content,'lxml')
        attr={'class':'tab-pane fade','id':'consolidated_valuation','role':'tabpanel'}
        div=soup.find('div',attrs=attr)
        data=div.find_all('div',attrs={'class':'value_txtfr'})
        for t in data:     
            temp=np.append(temp,[t.text])
        temp1=temp[11].split()
        values=np.array([temp[1],temp[3],temp[5],temp[8],temp[9],temp1[9],temp1[4],temp1[6],temp1[8]])
        series=[]
        for i in values:
            try:
                series.append(float(i))
            except(ValueError):
                series.append(np.NaN)
        return series
    except:
        series=[]
        for i in range(0,9):
            series.append(np.NaN)
        return series
        
#pe,Div,IndPe,pb,DivY,deliv1D,deliv3D,deliv5D,deliv8D 

In [ ]:
page=requests.get("https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/abrasives.html")
soup=bs(page.content,'lxml') #creating soup object to access links of all the industries

attr={'class':'colLft'}
section=soup.find('section',attrs=attr)#Scanning for section that contains list of all links


attr2={'href':re.compile("/stocks/marketstats/industry-classification/nse/")}
links=section.find_all('a',attrs=attr2)#storing all href containing links after parsing isdone

IndustryLinks=np.append(IndustryLinks,'https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/abrasives.html')
IndustryNames=np.append(IndustryNames,'abrasives')
link1='https://www.moneycontrol.com'

In [ ]:
i=1
for link in links:
    IndustryLinks=np.append(IndustryLinks,link1+str(link.get('href')))
    temp1=IndustryLinks[i].split('nse/')
    temp2=temp1[1].split('.html')
    IndustryNames=np.append(IndustryNames,temp2[0])
    i=i+1
#Extracting all industry links from html doc and storing them in numpy array

In [ ]:

i=0
l=0
for k in IndustryLinks:    
    try:
        page=requests.get(k)
        soup=bs(page.content,'lxml')
        t=IndustryNames[l].split('-')
    
        consent=True  #Purpose of consent is to check if that industry link leads to company links containing that industry name
                  # for example in agriculture industry names of it dont appear in company link in that case consent is left
                  # to be True so that the appropriate changes are made to parsing condition.
    
        for link in soup.findAll('a', attrs={'href': re.compile("https://www.moneycontrol.com/india/stockpricequote/"+t[0])}):
        
            try:
                CompaniesLinks=np.append(CompaniesLinks,link.get('href'))
                temp=CompaniesLinks[i].split('/')
                name=temp[6]+'('+temp[7]+')'
                IndustryName=temp[5]
            except:
                continue
            
       #pe,Div,Indpe,pb,DivY,deliv1D,deliv3D,deliv5D,deliv8D
            row=np.array(terms(CompaniesLinks[i]))
        
        #Gathering values of all parameters and storing them in their respective arrays corresponding to their companies and industries
        
            try:
                beta=np.append(beta,Beta(CompaniesLinks[i]))
            except:
                beta=np.append(beta,np.NaN)
            pe=np.append(pe,row[0])
            div=np.append(div,row[1])
            indpe=np.append(indpe,row[2])
            pb=np.append(pb,row[3])
            divy=np.append(divy,row[4])
            deliv1d=np.append(deliv1d,row[5])
            deliv3d=np.append(deliv3d,row[6])
            deliv5d=np.append(deliv5d,row[7])
            deliv8d=np.append(deliv8d,row[8])
        
            CompanyNames=np.append(CompanyNames,name)
            AllIndustries=np.append(AllIndustries,IndustryName)
        
            i+=1
            consent=False
        
    
        if(consent):
        
            page=requests.get(k)
            soup=bs(page.content,'lxml')
        
            span=soup.find_all('span',attrs={'class':'gld13 disin'})
        
            a=[span]
            t=str(a[0]).split('a href="')
        
            for i in range(len(t)):
                #Extracting link ofcompany from html doc by splitting data
                if(i%2==1):
                    a=t[i].split('"')
                    for j in a:
                        if(str('https://www.moneycontrol.com/india/stockpricequote/') in str(j)):
                        
                            CompaniesLinks=np.append(CompaniesLinks,j)
                            temp=CompaniesLinks[i].split('/')
                            name=temp[6]+'('+temp[7]+')'
                            IndustryName=temp[5]                            
                        
                        
                        #Gathering values of all parameters and storing them in their respective arrays corresponding to their companies and industries
                            try:
                                beta=np.append(beta,Beta(CompaniesLinks[i]))
                            except:
                                beta=np.append(beta,np.NaN)
                            
                            pe=np.append(pe,row[0])
                            div=np.append(div,row[1])
                            indpe=np.append(indpe,row[2])
                            pb=np.append(pb,row[3])
                            divy=np.append(divy,row[4])
                            deliv1d=np.append(deliv1d,row[5])
                            deliv3d=np.append(deliv3d,row[6])
                            deliv5d=np.append(deliv5d,row[7])
                            deliv8d=np.append(deliv8d,row[8])
        
                            CompanyNames=np.append(CompanyNames,name)
                            AllIndustries=np.append(AllIndustries,IndustryName)
                        i+=1
        l+=1
        print("Number of industries left:",(len(IndustryLinks)-l))
    except:
        print('Error in Industry:',j)
        continue
        
        
    

# Creating Dataframe

In [ ]:
df['Industry']=AllIndustries
df['Company']=CompanyNames
df['Beta']=beta
df['P/E']=pe
df['Dividend(%)']=div
df['Industry P/E']=indpe
df['P/B']=pb
df['Dividend Yield(%)']=divy
df['Deliverables(1D)']=deliv1d
df['Deliverables(3D)']=deliv3d
df['Deliverables(5D)']=deliv5d
df['Deliverables(8D)']=deliv8d

In [ ]:
df

In [ ]:
df=df.drop_duplicates(subset='Company',keep='first')#Removes all duplicates based on company name 

In [ ]:
df